In [1]:
import os
import sys
sys.path.append('/workspace/Documents')  ### remove this if not needed!
import numpy as np
import pandas as pd
from tqdm import tqdm
import random
from pathlib import Path
import nibabel as nb
import time

import argparse
from einops import rearrange
from natsort import natsorted
from madgrad import MADGRAD

import torch
import torch.backends.cudnn as cudnn
 
from cineCMR_SAM.utils.model_util import *
from cineCMR_SAM.segment_anything.model import build_model 
from cineCMR_SAM.utils.save_utils import *
from cineCMR_SAM.utils.config_util import Config
from cineCMR_SAM.utils.misc import NativeScalerWithGradNormCount as NativeScaler

from cineCMR_SAM.train_engine import train_loop

import cineCMR_SAM.dataset.build_CMR_datasets as build_CMR_datasets
import cineCMR_SAM.functions_collection as ff
import cineCMR_SAM.get_args_parser as get_args_parser

### define parameters for this experiment

In [2]:
# set experiment-specific parameters
main_path = '/mnt/camca_NAS/SAM_for_CMR/'  # replace with your own path
trial_name = 'cineCMR_sam_github'
output_dir = os.path.join(main_path, 'models', trial_name) # replace with your own path
text_prompt = True # whether we need to input text prompt to specify the view types (LAX or SAX). True or False. default = True
box_prompt = False # whether we have the bounding box for myocardium defined by the user. None means no box, 'one' means one box at ED and 'two' means two boxes at ED and ES

pretrained_model = None # define your pre-trained model if any
start_epoch = 1
total_training_epochs = 1 # define total number of epochs

# preload the text prompt feature (it's the output of a CLIP model when I input "LAX" or "SAX" into it)
sax_text_prompt_feature = np.load('/mnt/camca_NAS/SAM_for_CMR/data/text_prompt_clip/sax.npy')
lax_text_prompt_feature = np.load('/mnt/camca_NAS/SAM_for_CMR/data/text_prompt_clip/lax.npy')

# also define the original SAM model, replace with your own path (you can easily download the original SAM model from online)
original_sam = os.path.join( "/mnt/camca_NAS/SAM_for_CMR/", 'models/pretrained_sam/sam_vit_h_4b8939.pth') 

args = get_args_parser.get_args_parser(text_prompt = text_prompt, box_prompt = box_prompt, pretrained_model = pretrained_model, original_sam = original_sam, start_epoch = start_epoch, total_training_epochs = total_training_epochs)
args = args.parse_args([])

# some other settings
cfg = Config(args.config)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cudnn.benchmark = True

### define the training dataset (from SAX or/and LAX)

In [3]:
# define SAX training data
patient_list_file_sax = os.path.join(main_path,'models/cineCMR_sam_github/patient_list_sax.xlsx')
patient_index_list = np.arange(0,1,1)
dataset_train_sax = build_CMR_datasets.build_dataset(
        args,
        view_type = 'sax',
        patient_list_file = patient_list_file_sax, 
        index_list = patient_index_list, 
        text_prompt_feature = sax_text_prompt_feature,
        only_myo = True, 
        shuffle = True, 
        augment = True)

# define LAX training data
patient_list_file_lax = os.path.join(main_path,'models/cineCMR_sam_github/patient_list_lax.xlsx')
patient_index_list = np.arange(0,1,1)
dataset_train_lax = build_CMR_datasets.build_dataset(
        args,
        view_type = 'lax',
        patient_list_file = patient_list_file_lax, 
        index_list = patient_index_list, 
        text_prompt_feature = lax_text_prompt_feature,
        only_myo = True, 
        shuffle = True, 
        augment = True)

dataset_train = [dataset_train_sax, dataset_train_lax]

'''Set up data loader for training and validation set'''
data_loader_train = []
for i in range(len(dataset_train)):
    data_loader_train.append(torch.utils.data.DataLoader(dataset_train[i], batch_size = 1, shuffle = False, pin_memory = True, num_workers = 0))


### load pre-trained SAM model and set parameters

In [4]:
# set model
model = build_model(args, device)

# set freezed and trainable keys
train_keys = []
freezed_keys = []
        
# load pretrained sam model vit_h
if args.model_type.startswith("sam"):
    if args.resume.endswith(".pth"):
        with open(args.resume, "rb") as f:
            state_dict = torch.load(f)
        try:
            model.load_state_dict(state_dict)
        except:
            if args.vit_type == "vit_h":
                new_state_dict = load_from(model, state_dict, args.img_size,  16, [7, 15, 23, 31])
               
            model.load_state_dict(new_state_dict)
        
        # freeze original SAM layers
        freeze_list = [ "norm1", "attn" , "mlp", "norm2"]  
                
        for n, value in model.named_parameters():
            if any(substring in n for substring in freeze_list):
                freezed_keys.append(n)
                value.requires_grad = False
            else:
                train_keys.append(n)
                value.requires_grad = True

## Select optimization method
optimizer = MADGRAD(model.parameters(), lr=args.lr) # momentum=,weight_decay=,eps=)
        
# Continue training model
if args.pretrained_model is not None:
    if os.path.exists(args.pretrained_model):
        print('loading pretrained model : ', args.pretrained_model)
        args.resume = args.pretrained_model
        finetune_checkpoint = torch.load(args.pretrained_model)
        model.load_state_dict(finetune_checkpoint["model"])
        optimizer.load_state_dict(finetune_checkpoint["optimizer"])
        torch.cuda.empty_cache()
else:
    print('new training\n')

Important! text prompt: True
Important! box prompt: False


/tmp/ipykernel_3530388/1542652002.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(f)


new training



In [5]:
training_log = []
model_save_folder = os.path.join(output_dir, 'models'); ff.make_folder([output_dir, model_save_folder])
log_save_folder = os.path.join(output_dir, 'logs'); ff.make_folder([log_save_folder])

for epoch in range(args.start_epoch, args.start_epoch + args.total_training_epochs):
        print('training epoch:', epoch)

        if epoch % args.lr_update_every_N_epoch == 0:
            optimizer.param_groups[0]["lr"] = optimizer.param_groups[0]["lr"] * args.lr_decay_gamma
        print('learning rate now:', optimizer.param_groups[0]["lr"])
        
        loss_scaler = NativeScaler()
            
        train_results = train_loop(
                model = model,
                data_loader_train  = data_loader_train,
                optimizer = optimizer,
                epoch = epoch, 
                loss_scaler = loss_scaler,
                args = args,
                inputtype = cfg.data.input_type)   
        
        loss, lossCE, lossDICE, sax_loss, sax_lossCE, sax_lossDICE, lax_loss, lax_lossCE, lax_lossDICE = train_results       
            
        print('in epoch: ', epoch, ' training average_loss: ', loss, ' average_lossCE: ', lossCE, ' average_lossDICE: ', lossDICE, ' sax_loss: ', sax_loss, ' sax_lossCE: ', sax_lossCE, ' sax_lossDICE: ', sax_lossDICE, ' lax_loss: ', lax_loss, ' lax_lossCE: ', lax_lossCE, ' lax_lossDICE: ', lax_lossDICE)
    
        # on_epoch_end:
        for k in range(len(dataset_train)):
            dataset_train[k].on_epoch_end()
    
        if  epoch % args.save_model_file_every_N_epoch == 0 or (epoch + 1) == args.start_epoch + args.total_training_epochs:
            checkpoint_path = os.path.join(model_save_folder,  'model-%s.pth' % epoch)
            to_save = {
                        'model': model.state_dict(),
                        'optimizer': optimizer.state_dict(),
                        'epoch': epoch,
                        'scaler': loss_scaler.state_dict(),
                        'args': args,}
            torch.save(to_save, checkpoint_path)

        training_log.append([epoch, optimizer.param_groups[0]["lr"], train_results[0], train_results[1], train_results[2], train_results[3], train_results[4], train_results[5], train_results[6], train_results[7], train_results[8]])
        df = pd.DataFrame(training_log, columns=['epoch', 'lr','average_loss', 'average_lossCE', 'average_lossDICE', 'sax_loss', 'sax_lossCE', 'sax_lossDICE', 'lax_loss', 'lax_lossCE', 'lax_lossDICE'])
        df.to_excel(os.path.join(log_save_folder, 'training_log.xlsx'), index=False)

training epoch: 1
learning rate now: 0.0001
in train loop we have turn_zero_seg_slice_into:  10
in training current slice type:  sax


/workspace/Documents/cineCMR_SAM/utils/misc.py:252: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self._scaler = torch.cuda.amp.GradScaler()


in dataset_SAX, patient_id is:  ID_0002


/workspace/Documents/cineCMR_SAM/train_engine.py:50: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Epoch: [1]  [ 0/11]  eta: 0:00:24  lr: 0.000100  loss1: 5.8597 (5.8597)  time: 2.2047  data: 1.1594  max mem: 18854
in dataset_SAX, patient_id is:  ID_0002
in dataset_SAX, patient_id is:  ID_0002
in dataset_SAX, patient_id is:  ID_0002
in dataset_SAX, patient_id is:  ID_0002
in dataset_SAX, patient_id is:  ID_0002
in dataset_SAX, patient_id is:  ID_0002
in dataset_SAX, patient_id is:  ID_0002
in dataset_SAX, patient_id is:  ID_0002
in dataset_SAX, patient_id is:  ID_0002
in dataset_SAX, patient_id is:  ID_0002
Epoch: [1]  [10/11]  eta: 0:00:00  lr: 0.000100  loss1: 5.0624 (4.6847)  time: 0.5433  data: 0.3333  max mem: 18854
Epoch: [1] Total time: 0:00:05 (0.5436 s / it)
in training current slice type:  lax
in dataset_SAX, patient_id is:  ID_0085
Epoch: [1]  [0/1]  eta: 0:00:00  lr: 0.000100  loss1: 4.9009 (4.6050)  time: 0.6595  data: 0.5343  max mem: 18854
Epoch: [1] Total time: 0:00:00 (0.6605 s / it)
in epoch:  1  training average_loss:  4.605008602142334  average_lossCE:  4.1768759